## Elliptic Curve Diffie–Hellman Key Exchange:

#### Sources:
[ECC: a gentle introduction](andrea.corbellini.name/2015/05/23/elliptic-curve-cryptography-finite-fields-and-discrete-logarithms)\
(https://github.com/andreacorbellini/ecc/blob/master/scripts/ecdhe.py)

In [7]:
from ecc.scripts.ecdhe import *
from math import sqrt

In [23]:
#p - large prime
curve = EllipticCurve('test',p=29,a=-1,b=1,g=None,n=None,h=None)

#https://mathworld.wolfram.com/SingularPoint.html
assert 4*curve.a**3+27*curve.b**2 != 0

#calculating N, n, h=N/n 
N_list=[]
for G in P_list:
    P=G
    N_set=set()
    for _ in range(10**3):
        N_set.update([P])
        P=point_add(curve,P,G)
    N_list.append(len(N_set))
    
N=max(N_list) #N=37
n=N
h=1 # h=N/n

curve = EllipticCurve('test',p=curve.p
                      ,a=curve.a,b=curve.b,
                      g=None,
                      n=n,h=h)
#calculating G(x,y)
y2=lambda x: (x**3+a*x+b) % p
#get all_points
P_list={ ( x%p , int(sqrt(y2(x)))) for x in range(10**3) if sqrt(y2(x)) in range(p) }

#get random point
G=random.sample(P_list,1)[0]
assert scalar_mult(curve,h,G)!=(0,0)

In [24]:
curve = EllipticCurve(
    'test',
    # Field characteristic.
    p=29,
    # Curve coefficients.
    a=-1,
    b=1,
    # Base point.
    g=G,
    # Subgroup order.
    n=37,
    # Subgroup cofactor. h=N/n
    h=1,
)

## Key exchange

In [25]:
print('Curve:', curve.name)

# Alice generates her own keypair.
alice_private_key, alice_public_key = make_keypair(curve)
print("Alice's private key:", alice_private_key)
print("Alice's public  key:",alice_public_key)

# Bob generates his own key pair.
bob_private_key, bob_public_key = make_keypair(curve)
print("Bob's private key:", bob_private_key)
print("Bob's public  key:",bob_public_key)

# Alice and Bob exchange their public keys and calculate the shared secret.
s1 = scalar_mult(curve,alice_private_key, bob_public_key)
s2 = scalar_mult(curve,bob_private_key, alice_public_key)
assert s1 == s2

print('Shared secret:',s1)

Curve: test
Alice's private key: 23
Alice's public  key: (27, 16)
Bob's private key: 1
Bob's public  key: (28, 1)
Shared secret: (27, 16)


In [33]:
#"Now that Alice and Bob have obtained the shared secret, they can exchange data with symmetric encryption."
#XOR toy example below.

#Alice
msg_in=random.randrange(0,p)
c=msg_in ^ s1[0]

#Bob
msg_out = c ^ s2[0]

assert msg_in==msg_out